# <center> GRAD5100 - Final project </center>
## Part Three
## Yoobee Lim

In [1]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os
import numpy as np

In [2]:
username='user'
password='grad5100user'
awsresource='database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com'
dbname='topChef'
port = 3306

dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [3]:
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

In [4]:
tables = run_sql("show tables;", engine)
for x in tables:
    print(x)

('challengedescriptions',)
('challengewins',)
('chefdetails',)
('episodeinfo',)
('judges',)
('rewards',)


# 1. Create Tables

## i. chefdetails

In [5]:
with engine.connect() as conn:
    df_chefdetails = pd.read_sql(
       sqla.text("SELECT * FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%' ;"
                ), conn
   )
df_chefdetails.head()

,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
0,0,Richard Blais,Richard B.,None,None,None,38.0,All Stars: New York,8,US,1,None,Owner,Male
1,1,Mike Isabella,Mike I.,None,None,None,35.0,All Stars: New York,8,US,2,None,Executive Chef/Owner,Male
2,2,Antonia Lofaso,Antonia L.,None,None,None,34.0,All Stars: New York,8,US,3,None,Executive Chef,Female
3,3,Tiffany Derry,Tiffany D.,None,None,None,27.0,All Stars: New York,8,US,4,POC,Executive Chef,Female
4,4,Carla Hall,Carla H.,None,None,None,46.0,All Stars: New York,8,US,5,POC,Chef/Owner,Female


### 2-ii Include only the columns of interest

In [6]:
df_chefdetails.columns

Index(['num', 'name', 'chef', 'hometown', 'city', 'state', 'age', 'season',
       'seasonNumber', 'series', 'placement', 'personOfColor', 'occupation',
       'gender'],
      dtype='object')

In [7]:
df_chefdetails = df_chefdetails[['season','seasonNumber','name','placement','gender']]
df_chefdetails.head()

,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female


## ii. judges 

In [8]:
with engine.connect() as conn:
    df_judges = pd.read_sql(
       sqla.text("SELECT * FROM judges WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%' ;"
                ), conn
   )
df_judges.head()

,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
0,0,All Stars: New York,8,US,1,Quickfire,Team,Tom Colicchio,None,Top Chef Judge
1,1,All Stars: New York,8,US,1,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations
2,2,All Stars: New York,8,US,2,Quickfire,Individual,Joe Jonas,None,None
3,3,All Stars: New York,8,US,2,Elimination,Team,Katie Lee,None,Beat Bobby Flay Judge
4,4,All Stars: New York,8,US,3,Quickfire,Team,David Chang,None,None


In [9]:
# In judges table there was weired data in season that has space : 'Charleston '
# To handle this ...

df_judges['season'] = df_judges['season'].str.replace('Charleston ', 'Charleston')

### 2- ii. Include only the columns of interest

In [10]:
df_judges=df_judges[['season','seasonNumber','episode','challengeType','guestJudge','competedOnTC']]
df_judges.head()

,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None


## 3. New table

In [11]:
df_season=df_chefdetails[['season','seasonNumber']].drop_duplicates().reset_index(drop=True)
df_season.sort_values(by='seasonNumber')

,season,seasonNumber
16,San Francisco,1
11,Los Angeles,2
12,Miami,3
5,Chicago,4
14,New York,5
10,Las Vegas,6
6,D.C.,7
0,All Stars: New York,8
18,Texas,9
17,Seattle,10


# Answer the Questions

a. Among all chef contestants, how many are male and how many are female?

In [12]:
gender = df_chefdetails.gender.value_counts()
gender

Male      170
Female    152
Name: gender, dtype: int64

b. Among all winners (placement==1) how many are male and how many are female?

In [13]:
win = df_chefdetails[df_chefdetails['placement']==1]['gender'].value_counts()
win

Male      14
Female     6
Name: gender, dtype: int64

c. Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many
are female?

In [14]:
top3 = df_chefdetails[df_chefdetails['placement'].isin([1,2,3])]['gender'].value_counts()
top3

Male      36
Female    25
Name: gender, dtype: int64

d. Compare the number of times a female was placed in the top 3 with the number of times
a female won. Does this number seem unusual? That is, are women who place in the
top 3 less likely than expected to ultimately win the competition? Why or why not?

In [15]:
p = win['Female'] / top3['Female']
print(f'Probability women who place in the top3 win the competition: {p*100}%')


Probability women who place in the top3 win the competition: 24.0%


Women who place in the top 3 less likely than expected to ultimately win the competition. <br>
Since the probability that women who place in the top3 win the competition is just 24%

# 4. Make a table 
with one row for each first place winner containing the winner’s name,
the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they
served as a guest judge or not.

In [16]:
df_onTC = df_judges[df_judges['competedOnTC']=='Yes']
onTC_judge = set(df_onTC.guestJudge)

In [17]:
df_win = df_chefdetails[df_chefdetails['placement']==1]
df_win = df_win[['name','season']]

In [18]:
df_win['served_as_judge'] = np.nan
df_win=df_win.reset_index(drop=True)

In [19]:
for i in range( len(df_win)):
    if df_win.at[i, 'name'] in onTC_judge:
#         print(df_win.at[i, 'name'])
        df_win.at[i, 'served_as_judge'] = 'Yes'
    else:
        df_win.at[i, 'served_as_judge'] = 'No'

In [20]:
df_win

,name,season,served_as_judge
0,Richard Blais,All Stars: New York,Yes
1,Melissa King,All-Stars L.A.,Yes
2,Mei Lin,Boston,No
3,Jeremy Ford,California,No
4,Brooke Williamson,Charleston,Yes
5,Stephanie Izard,Chicago,Yes
6,Kevin Sbraga,D.C.,No
7,Joseph Flamm,Colorado,No
8,Kah-wai Lo,Houston,No
9,Kelsey Barnard Clark,Kentucky,Yes


# 5. Find all contestants
who were not first place winners but did serve as a guest judge at some point. Make sure to only list each name once.

In [21]:
df_notwin = df_chefdetails[df_chefdetails['placement']!=1]

In [22]:
notwin_list=list(set(df_notwin.name))
win_list = set(df_win.name)
judge_list = set(df_judges.guestJudge)

In [23]:
notwin_judge = []

for i in range(0,len(notwin_list)-1):
    if notwin_list[i] in win_list:
#         print(notwin_list[i])
        continue
    elif notwin_list[i] in judge_list:
        notwin_judge.append(notwin_list[i])

all contestants who were not first place winners but did serve as a guest judge at some point :

In [24]:
notwin_judge

['Amar Santana',
 'Dawn Burrell',
 'Karen Akunowicz',
 'Sheldon Simeon',
 'Spike Mendelsohn',
 'Stephen Asprinio',
 'Sam Talbot',
 'Miguel Morales',
 'Edward Lee',
 'Nyesha Arrington',
 'Kwame Onwuachi',
 'Bryan Voltaggio',
 'Tiffany Derry',
 'Nina Compton',
 'Nini Nguyen',
 'Lee Anne Wong',
 'Adrienne Cheatham',
 'Dale Talde',
 'Antonia Lofaso',
 'Josie Smith-Malave',
 'Shota Nakajima',
 'Mike Isabella',
 'Carrie Baird',
 'Stefan Richter',
 'Gregory Gourdet']